In [ ]:
import logging
logging.basicConfig(level=logging.INFO)

import pickle 
import numpy as np

from openfold.data import data_pipeline, feature_pipeline
from openfold.utils.tensor_utils import tensor_tree_map, dict_multimap
from openfold.config import model_config
from openfold.utils.loss import compute_contact_ca

In [ ]:
ss_file = '/Users/chenceshi/Downloads/Chrome Downloads/ss_annotation_31885.pkl'
ss_dict = {}
with open(ss_file, 'rb') as fin:
    second_structure_data = pickle.load(fin)
logging.warning(f"get {len(second_structure_data)} second structure data")
for ss in second_structure_data:
    ss_dict[ss['tag']] = ss['ss3']
data_pipeline = data_pipeline.DataPipeline(ss_dict)

In [ ]:
config = model_config(name='initial_training', train=True)
feature_pipeline = feature_pipeline.FeaturePipeline(config.data)

In [ ]:
path = '/Users/chenceshi/Downloads/1a0aA00.pdb'
data = data_pipeline.process_pdb(pdb_path=path)
feats = feature_pipeline.process_features(data, 'train')

In [ ]:
cycle_no = 0
fetch_cur_batch = lambda t: t[..., cycle_no]
cur_feats = tensor_tree_map(fetch_cur_batch, feats)

contact = compute_contact_ca(
    cur_feats["all_atom_positions"],
    cur_feats["all_atom_mask"],
    cutoff=12
)
contact_ = contact[:63,:63].numpy()

In [ ]:
beautiful_out = []
for line in contact_:
    tmp = []
    for x_ in line:
        tmp.append(str(x_))
    beautiful_out.append(''.join(tmp))
print('\n'.join(beautiful_out))


In [52]:
cur_feats

{'aatype': tensor([12, 11,  1,  6, 15,  8, 11,  8,  0,  6,  5,  0,  1,  1,  2,  1, 10,  0,
         19,  0, 10,  8,  6, 10,  0, 15, 10,  9, 14,  0,  6, 17, 11,  5,  5,  2,
         19, 15,  0,  0, 14, 15, 11,  0, 16, 16, 19,  6,  0,  0,  4,  1, 18,  9,
          1,  8, 10,  5,  5,  2,  7, 15, 16,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
          0,  0,  